
# Sentinel-2 NDVI anomaly cube (z-scores)

This notebook mirrors the PRISM and gridMET worked examples but focuses on
streaming Sentinel-2 Level-2A data, computing NDVI, and standardizing the result
with z-scores so you can compare vegetation anomalies to climate drivers.


In [ ]:

from __future__ import annotations

import warnings

import cubo
import matplotlib.pyplot as plt
import xarray as xr

from cubedynamics import pipe, verbs as v


In [ ]:

LAT = 43.89
LON = -102.18
START = "2023-06-01"
END = "2024-09-30"

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    s2 = cubo.create(
        lat=LAT,
        lon=LON,
        collection="sentinel-2-l2a",
        bands=["B04", "B08"],
        start_date=START,
        end_date=END,
        edge_size=512,
        resolution=10,
        query={"eo:cloud_cover": {"lt": 40}},
    )

s2


In [ ]:

ndvi_z = (
    pipe(s2)
    | v.ndvi_from_s2(nir_band="B08", red_band="B04")
    | v.zscore(dim="time")
).unwrap()

ndvi_z


In [ ]:

(pipe(ndvi_z) | v.show_cube_lexcube(title="Sentinel-2 NDVI z-score", clim=(-3, 3)))

median_series = ndvi_z.median(dim=("y", "x"))
median_series.plot.line(x="time", ylabel="Median NDVI z-score", figsize=(9, 3))
plt.title("Spatial median NDVI anomaly")
plt.show()



You can compare this NDVI anomaly cube to PRISM or gridMET anomaly cubes with
`v.correlation_cube` so that vegetation and climate diagnostics share the same
pixels.
